# MPI parallelisation

We will need to install two packages:

- The python MPI package:

```
conda install mpi4py
```

- The actual MPI library:

```
conda install openmpi
```

### References: 
https://mpi4py.readthedocs.io/en/stable/tutorial.html

https://education.molssi.org/parallel-programming/03-distributed-examples-mpi4py/index.html


## 1. Basic python script:

Let's create a .py script that writes "Hello World":


```python
if __name__ == "__main__":

    print("Hello World!")
    
```

To execute it in a terminal:

```
wladimir$ python example_mpi.py

Hello World!
```

To execute it in a terminal:

```
wladimir$ python example1.py 

Hello World!

## 2. Getting Started with MPI

Let’s try running this code on multiple processes. This is done using the **mpiexec** command.

Many environments also provide an **mpirun** command, which usually - but not always - works the same way. 

Whenever possible, you should use mpiexec and not mpirun, in order to guarantee more consistent results.

### MPI - mpiexec vs mpirun

- MPI stands for **‘message passing interface’** and is a message passing standard which is designed to work on a variety of parallel computing architectures.


- The MPI standard defines how syntax and semantics of a library of routines. There are a number of implementations of this standard including OpenMPI, MPICH2, and MS MPI.


- The primary difference between mpiexec and mpirun is that mpiexec is defined as part of the MPI standard, while mpirun is not.


- Different implementations of MPI (i.e. OpenMPI, MPICH, MS MPI, etc.) are not guaranteed to implement mpirun, or might implement different options for mpirun.


- Technically, the MPI standard doesn’t actually require that MPI implementations implement mpiexec either, but the standard does at least describe guidelines for how mpiexec should work. Because of this, mpiexec is generally the preferred command.


The general format for launching a code on multiple processes is:

Let's create a **example2.py** script that writes "Hello World" and execute it in a terminal:


```
wladimir$ mpiexec -n 2 python example_mpi.py
Hello World!
Hello World! 
```

When you execute the above command, mpiexec launches 2 different instances of python example1.py simultaneously, which each print “Hello World!”.

Typically, as long as you have at least 2 processors on the machine you are running on, each process will be launched on a different processor; however, certain environment variables and optional arguments to mpiexec can change this behavior. Each process runs the code in example2.py independently of the others.

### MPI communicator:

It might not be obvious yet, but the processes mpiexec launches aren’t completely unaware of one another. The mpiexec adds each of the processes to an MPI communicator, which enables each of the processes to send and receive information to one another via MPI. The MPI communicator that spans all of the processes launched by mpiexec is called **MPI.COMM_WORLD.**

In mpi4py, communicators are class objects, and we can query information about them through their class functions. Edit example2.py so that it reads as follows: